In [1]:


import pandas as pd
import numpy as np
import re
import nltk
import pickle

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score


In [2]:
nltk.download('stopwords')
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wwwan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
fake_df=pd.read_csv(r'Fake.csv')
true_df=pd.read_csv(r'True.csv')

In [4]:
fake_df['label']=1
true_df['label']=0

In [5]:
df=pd.concat([fake_df,true_df],axis=0)
# suffle the data
df=df.sample(frac=1,random_state=42).reset_index(drop=True)

In [6]:
df.head()

,title,text,subject,date,label
0,Ben Stein Calls Out 9th Circuit Court: Committ...,"21st Century Wire says Ben Stein, reputable pr...",US_News,"February 13, 2017",1
1,Trump drops Steve Bannon from National Securit...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"April 5, 2017",0
2,Puerto Rico expects U.S. to lift Jones Act shi...,(Reuters) - Puerto Rico Governor Ricardo Rosse...,politicsNews,"September 27, 2017",0
3,OOPS: Trump Just Accidentally Confirmed He Le...,"On Monday, Donald Trump once again embarrassed...",News,"May 22, 2017",1
4,Donald Trump heads for Scotland to reopen a go...,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",politicsNews,"June 24, 2016",0


In [7]:
import pandas as pd

# Sample DataFrame
df1= pd.DataFrame({
    'name': ['A', 'B', 'C', 'D', 'E'],
    'score': [10, 20, 30, 40, 50]
})


In [8]:
shuffled1 = df1.sample(frac=1)
shuffled2 = df1.sample(frac=1)

print(shuffled1)
print(shuffled2)


  name  score
0    A     10
4    E     50
2    C     30
3    D     40
1    B     20
  name  score
4    E     50
0    A     10
3    D     40
1    B     20
2    C     30


In [9]:
shuffled1 = df1.sample(frac=1, random_state=42)
shuffled2 = df1.sample(frac=1, random_state=42)

print(shuffled1)
print(shuffled2) ## now understood the mean of the random state 


  name  score
1    B     20
4    E     50
2    C     30
0    A     10
3    D     40
  name  score
1    B     20
4    E     50
2    C     30
0    A     10
3    D     40


In [10]:
df.describe()

,label
count,44898.000000
mean,0.522985
std,0.499477
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   label    44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.7+ MB


In [12]:
df.isnull().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [13]:
df=df.drop(['title','subject','date'],axis=1)

In [14]:
df

,text,label
0,"21st Century Wire says Ben Stein, reputable pr...",1
1,WASHINGTON (Reuters) - U.S. President Donald T...,0
2,(Reuters) - Puerto Rico Governor Ricardo Rosse...,0
3,"On Monday, Donald Trump once again embarrassed...",1
4,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",0
...,...,...
44893,,1
44894,LONDON/TOKYO (Reuters) - British Prime Ministe...,0
44895,BERLIN (Reuters) - Chancellor Angela Merkel sa...,0
44896,Jesus f*cking Christ our President* is a moron...,1


In [15]:
def preprocess_text(text):
    # Remove non-alphabet characters (replace with space)
    text = re.sub(r"[^a-zA-Z]", " ", text)
    # Convert to lowercase
    text = text.lower()
    # Tokenize into words
    words = text.split()
    # Remove stopwords and apply stemming
    words = [ps.stem(word) for word in words if word not in stop_words]
    # Join back into single string
    return " ".join(words)


In [16]:
df['text'][0]

'21st Century Wire says Ben Stein, reputable professor from, Pepperdine University (also of some Hollywood fame appearing in TV shows and films such as Ferris Bueller s Day Off) made some provocative statements on Judge Jeanine Pirro s show recently. While discussing the halt that was imposed on President Trump s Executive Order on travel. Stein referred to the judgement by the 9th Circuit Court in Washington state as a  Coup d tat against the executive branch and against the constitution.  Stein went on to call the Judges in Seattle  political puppets  and the judiciary  political pawns. Watch the interview below for the complete statements and note the stark contrast to the rhetoric of the leftist media and pundits who neglect to note that no court has ever blocked any Presidential orders in immigration in the past or discuss the legal efficacy of the halt or the actual text of the Executive Order.READ MORE TRUMP NEWS AT: 21st Century Wire Trump FilesSUPPORT OUR WORK BY SUBSCRIBING &

In [17]:
preprocess_text('Breaking News: Tech giant “NeoCom®” announces a $5.2 billion acquisition of @CloudZen Inc.—a major player in A.I. & data security. The deal, signed at 3:45 p.m. (GMT+0), is expected to close by Q4 2025! #TechNews #M&A')

'break news tech giant neocom announc billion acquisit cloudzen inc major player data secur deal sign p gmt expect close q technew'

In [18]:
preprocess_text('21st Century Wire says Ben Stein, reputable professor from, Pepperdine University (also of some Hollywood fame appearing in TV shows and films such as Ferris Bueller s Day Off) made some provocative statements on Judge Jeanine Pirro s show recently. While discussing the halt that was imposed on President Trump s Executive Order on travel. Stein referred to the judgement by the 9th Circuit Court in Washington state as a  Coup d tat against the executive branch and against the constitution.  Stein went on to call the Judges in Seattle  political puppets  and the judiciary  political pawns. Watch the interview below for the complete statements and note the stark contrast to the rhetoric of the leftist media and pundits who neglect to note that no court has ever blocked any Presidential orders in immigration in the past or discuss the legal efficacy of the halt or the actual text of the Executive Order.READ MORE TRUMP NEWS AT: 21st Century Wire Trump FilesSUPPORT OUR WORK BY SUBSCRIBING & BECOMING A MEMBER @21WIRE.TV')

'st centuri wire say ben stein reput professor pepperdin univers also hollywood fame appear tv show film ferri bueller day made provoc statement judg jeanin pirro show recent discuss halt impos presid trump execut order travel stein refer judgement th circuit court washington state coup tat execut branch constitut stein went call judg seattl polit puppet judiciari polit pawn watch interview complet statement note stark contrast rhetor leftist media pundit neglect note court ever block presidenti order immigr past discuss legal efficaci halt actual text execut order read trump news st centuri wire trump filessupport work subscrib becom member wire tv'

In [19]:
df['text']=df['text'].apply(preprocess_text)

In [20]:
# define the features X and Y
X=df['text']
Y=df['label']

In [21]:
## spliting the features in training and text set
X_train,X_test,y_train,y_test=train_test_split(X,Y,random_state=42,test_size=0.25)

In [22]:
# train tf-idf on the full dataset
vectorizer=TfidfVectorizer(max_features=5000)
X_train_tfidf=vectorizer.fit_transform(X_train) ## train on entire dataset


In [23]:
X_test_tfidf=vectorizer.transform(X_test)

### train a naive bayes model

In [24]:
model=MultinomialNB()
model.fit(X_train_tfidf,y_train)

MultinomialNB()

In [25]:
## model accuracy 
y_pred=model.predict(X_test_tfidf)
accuracy=accuracy_score(y_test,y_pred)
print(f"accuracy of model is {accuracy*100:.2f}%")

accuracy of model is 92.51%


In [26]:
## save the TF_IDF vectorizer 
with open('vectorizer.pkl','wb') as file:
    pickle.dump(vectorizer,file)
# save model
with open("model.pkl",'wb') as file:
    pickle.dump(model,file) 

In [27]:
## load the TF-IDF vectorizer 
vector_form=pickle.load(open('vectorizer.pkl','rb'))
# load the model
load_model=pickle.load(open('model.pkl','rb'))

In [28]:
# Function to Predict Fake News
def fake_news(news):
    news = preprocess_text(news)  # Use the same preprocessing
    vectorized_news = vector_form.transform([news])  #  Transform using trained TF-IDF
    prediction = load_model.predict(vectorized_news)
    return "Fake News" if prediction == 1 else "Real News"

# Test with an external news article
new_article = """The Reserve Bank of India kept the repo rate unchanged at 6.5% in its latest monetary policy announcement."""
result = fake_news(new_article)
print("Prediction:", result)


Prediction: Real News


In [29]:
new_article="""NASA confirms Earth will go completely dark for 6 days in December due to a solar storm."""
result=fake_news(new_article)
print(f"prediction : {result}")

prediction : Fake News
